## This Notebook is for a Capstone

This will be the primary use of said notebook. Please be aware

In [2]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import requests


In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Time to Segment and Classify Toronto

### First
Scrape the Postal Code Data from the Wiki    
And throw it into a dataframe

In [4]:
#Puts Wikiscrape into DF
pc_tor = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(pc_tor.content,'lxml')
codes = soup.find_all('table')[0]
zips_df = pd.read_html(str(codes), header = 0)
zips_df= zips_df[0]

### Second
Remove not assigned Boroughs

In [5]:
zips_df= zips_df.where(zips_df['Borough']!= 'Not assigned').dropna()

### Third
Group by Postal Code and make the data neat

In [6]:
def bump_set_spike(ball):
    ball = set(ball)
    point = ''
    for play in range(len(ball)):
        if play != 0: point += ', '
        point += ball.pop()
    return point
print(bump_set_spike(['1','2','3']))

3, 1, 2


In [7]:
Gzips = zips_df.groupby(zips_df['Postcode'], as_index=True)

In [8]:
b_zips = Gzips['Borough'].apply(lambda x:bump_set_spike(x)).to_frame()
n_zips = Gzips['Neighbourhood'].apply(lambda x:bump_set_spike(x)).to_frame()
j_zips = b_zips.join(n_zips)

### Finally
Take care  of Not assigned Neighbourhood(s)

In [9]:
aHold = j_zips[j_zips.Neighbourhood.str.contains('Not assigned')]
aHold

,Borough,Neighbourhood
Postcode,,
M7A,Queen's Park,Not assigned


In [10]:
j_zips.loc[aHold.index[0],'Neighbourhood'] = aHold.Borough[0]

In [11]:
j_zips.shape

(103, 2)

# Geocoder Section
Geocoding the Post codes

I Don't wish to put in an API key so I will just pull .CSV

In [1]:
!wget -O postcode_data.csv http://cocl.us/Geospatial_data

In [61]:
geopost = pd.read_csv('postcode_data.csv')
geopost.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [71]:
post_lola = j_zips.join(geopost.set_index('Postal Code'))
post_lola.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Oakridge, Golden Mile",43.711112,-79.284577
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
